In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
import sys

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
print('SYSTEM Version: ', sys.version)
print('PyTorch Version: ', torch.__version__)
# print('Torchvision Version: ', torchvision.__version__)
print('Numpy Version: ', np.__version__)
print('Pandas Version: ', pd.__version__)

SYSTEM Version:  3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
PyTorch Version:  2.1.2
Numpy Version:  1.26.4
Pandas Version:  2.2.1


# Pytorch Dataset
import data

In [3]:
data_dir = '/kaggle/input/cards-image-datasetclassification/train'


In [4]:
class PlayingCardDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def clases(self):
        return self.data.classes
    

In [5]:
dataset = PlayingCardDataset(
    data_dir = '/kaggle/input/cards-image-datasetclassification/train'
)

In [6]:
len(dataset)

7624

In [7]:
image, label = dataset[6000]
print(label)
type(image)

41


PIL.Image.Image

In [8]:
target_to_class =  {v: k for k,v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

{0: 'ace of clubs', 1: 'ace of diamonds', 2: 'ace of hearts', 3: 'ace of spades', 4: 'eight of clubs', 5: 'eight of diamonds', 6: 'eight of hearts', 7: 'eight of spades', 8: 'five of clubs', 9: 'five of diamonds', 10: 'five of hearts', 11: 'five of spades', 12: 'four of clubs', 13: 'four of diamonds', 14: 'four of hearts', 15: 'four of spades', 16: 'jack of clubs', 17: 'jack of diamonds', 18: 'jack of hearts', 19: 'jack of spades', 20: 'joker', 21: 'king of clubs', 22: 'king of diamonds', 23: 'king of hearts', 24: 'king of spades', 25: 'nine of clubs', 26: 'nine of diamonds', 27: 'nine of hearts', 28: 'nine of spades', 29: 'queen of clubs', 30: 'queen of diamonds', 31: 'queen of hearts', 32: 'queen of spades', 33: 'seven of clubs', 34: 'seven of diamonds', 35: 'seven of hearts', 36: 'seven of spades', 37: 'six of clubs', 38: 'six of diamonds', 39: 'six of hearts', 40: 'six of spades', 41: 'ten of clubs', 42: 'ten of diamonds', 43: 'ten of hearts', 44: 'ten of spades', 45: 'three of clu

In [9]:
transform = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.ToTensor(),
])
dataset = PlayingCardDataset(data_dir, transform)

In [10]:
image, label = dataset[100]
print(label)
image.shape

0


torch.Size([3, 128, 128])

In [ ]:
Test1
